<a href="https://colab.research.google.com/github/omid-sakaki-ghazvini/Practices/blob/main/Academic_Success.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install Dependencies and Setup

<div style="direction:rtl">
<font color='green' size="5px">
 کتابخانه های مورد نیاز را نصب میکنیم
    </font>
</div>

In [ ]:
pip install xgboost lightgbm flaml

In [ ]:
#Data Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from flaml import AutoML


<div style="direction:rtl">
<font color='green' size="5px">
 از لینک زیر دیتاست را دانلود کرده و در پوشه هم مسیر همین ژوپیتر نوت بوک قرار دهید
    </font>
</div>

## https://www.kaggle.com/competitions/playground-series-s4e6/data

# 2. Load Data

<div style="direction:rtl">
<font color='green' size="5px">
توسط خط فرمان زیر، دیتا را فراخوانی میکنیم
    </font>
</div>

In [ ]:
df = pd.read_csv('/kaggle/input/playground-series-s4e6/train.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## 3.Data Analysis

In [ ]:
df['Target'].unique()

In [ ]:
df.groupby('Target')['Target'].count()

In [ ]:
train_df.describe().T

In [ ]:
fig, axes = plt.subplots(3, 2, sharex=True, figsize=(15,10))

sns.kdeplot(data=train_df ,x='Curricular units 2nd sem (approved)', hue='Target', ax=axes[0,0]);
axes[0,0].set_title('Curricular units 2nd sem (approved)');

sns.kdeplot(data=train_df ,x='Curricular units 2nd sem (grade)', hue='Target', ax=axes[0,1]);
axes[0,1].set_title('Curricular units 2nd sem (grade)');

sns.kdeplot(data=train_df ,x='Curricular units 1st sem (approved)', hue='Target', ax=axes[1,0]);
axes[1,0].set_title('Curricular units 1st sem (approved)');

sns.kdeplot(data=train_df ,x='Curricular units 1st sem (grade)', hue='Target', ax=axes[1,1]);
axes[1,1].set_title('Curricular units 1st sem (grade)');

sns.kdeplot(data=train_df ,x='Curricular units 2nd sem (evaluations)', hue='Target', ax=axes[2,0]);
axes[2,0].set_title('Curricular units 2nd sem (evaluations)');

sns.kdeplot(data=train_df ,x='Curricular units 1st sem (evaluations)', hue='Target', ax=axes[2,1]);
axes[2,1].set_title('Curricular units 1st sem (evaluations)');

In [ ]:
fig, axes = plt.subplots(1, 3, sharex=True, figsize=(10,5))

sns.scatterplot(data=train_df ,x='Curricular units 1st sem (approved)', y='Curricular units 2nd sem (approved)', hue='Target', ax=axes[0]);

sns.scatterplot(data=train_df ,x='Curricular units 1st sem (grade)', y='Curricular units 2nd sem (grade)', hue='Target', ax=axes[1]);

sns.scatterplot(data=train_df ,x='Curricular units 1st sem (evaluations)', y='Curricular units 2nd sem (evaluations)', hue='Target',ax=axes[2]);

## 4.Data preparation

In [ ]:
Target_names={'Graduate':0, 'Dropout':1, 'Enrolled':2}

X = train_df.drop(columns = ['id','Target'])
y = train_df['Target'].map(Target_names)

# 5. Split & Scale Data

<div style="direction:rtl">
<font color='green' size="5px">
 نرمالسازی دیتا
    </font>
</div>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
scaler_data = StandardScaler()

X_train_scaled = scaler_data.fit_transform(X_train)
X_test_scaled = scaler_data.fit_transform(X_test)

# 6. Build Models

In [ ]:
models = {
    'automl' : AutoML(task="classification", time_budget=100, n_jobs=-1, metric="accuracy"),
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(eval_metric='logloss', random_state=42),
    'AdaBoost': AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=100,min_samples_split=8,min_samples_leaf=4,random_state=42)),
    'GradientBoost' : GradientBoostingClassifier(loss='log_loss',n_estimators=100),
    'LGBM' : LGBMClassifier(random_state=42)

}

results = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]  # Probabilities for churn

    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred, output_dict=True)

    results[name] = {
        'Accuracy': accuracy,
        'Precision': classification_rep['1']['precision'],
        'Recall': classification_rep['1']['recall'],
        'F1': classification_rep['1']['f1-score']
    }

results_df = pd.DataFrame(results).T

# 7. Results

In [ ]:
results_df

In [ ]:
plt.figure(figsize=(10,6))

plt.title("Results")

sns.lineplot(results_df['Accuracy'])

plt.ylabel("Accuracy");